In [22]:
import requests
import json
import math
import pandas as pd
import numpy as np
import datetime


In [23]:
file_path = './clinicaltrials_dataframe.csv'
df_original = pd.read_csv(file_path)

<ipython-input-23-f0db41ed596b>:2: DtypeWarning: Columns (9,10,19,28,38,42,97,98,99,106,121,173,208,209,216,230,250,251,252,253,264) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(file_path)


In [24]:
print(df_original.columns.values)

['Acronym' 'AgreementOtherDetails' 'AgreementPISponsorEmployee'
 'AgreementRestrictionType' 'AgreementRestrictiveAgreement'
 'ArmGroupDescription' 'ArmGroupInterventionName' 'ArmGroupLabel'
 'ArmGroupType' 'AvailIPDComment' 'AvailIPDId' 'AvailIPDType'
 'AvailIPDURL' 'BaselineCategoryTitle' 'BaselineClassDenomCountGroupId'
 'BaselineClassDenomCountValue' 'BaselineClassDenomUnits'
 'BaselineClassTitle' 'BaselineDenomCountGroupId'
 'BaselineDenomCountValue' 'BaselineDenomUnits' 'BaselineGroupDescription'
 'BaselineGroupId' 'BaselineGroupTitle' 'BaselineMeasureCalculatePct'
 'BaselineMeasureDenomCountGroupId' 'BaselineMeasureDenomCountValue'
 'BaselineMeasureDenomUnits' 'BaselineMeasureDenomUnitsSelected'
 'BaselineMeasureDescription' 'BaselineMeasureDispersionType'
 'BaselineMeasureParamType' 'BaselineMeasurePopulationDescription'
 'BaselineMeasureTitle' 'BaselineMeasureUnitOfMeasure'
 'BaselineMeasurementComment' 'BaselineMeasurementGroupId'
 'BaselineMeasurementLowerLimit' 'BaselineMeas

In [25]:
df_original.info()  # Obtener información sobre tipos de datos y valores faltantes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Columns: 328 entries, Acronym to WhyStopped
dtypes: float64(41), object(287)
memory usage: 33.8+ MB


In [26]:
# Eliminar columnas donde más del 50% de los valores sean NaN
df_cleaned = df_original.dropna(axis=1, thresh=len(df_original) * 0.5)

# Mostrar las informacion de las columnas resultantes
print(df_cleaned.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 82 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ArmGroupDescription              10426 non-null  object 
 1   ArmGroupInterventionName         11067 non-null  object 
 2   ArmGroupLabel                    11662 non-null  object 
 3   ArmGroupType                     9759 non-null   object 
 4   BriefSummary                     13501 non-null  object 
 5   BriefTitle                       13501 non-null  object 
 6   CompletionDate                   12888 non-null  object 
 7   CompletionDateType               12764 non-null  object 
 8   Condition                        13500 non-null  object 
 9   ConditionAncestorId              12202 non-null  object 
 10  ConditionAncestorTerm            12202 non-null  object 
 11  ConditionBrowseBranchAbbrev      13253 non-null  object 
 12  ConditionBrowseBra

In [27]:
# Crea df_cleaned como un DataFrame independiente
df_cleaned = df_cleaned.copy()

# Lista de columnas a conservar
columns_to_keep = [
    "WhyStopped", "OutcomeAnalysisPValue", "ReferenceCitation", "ReferencePMID"
]

# Añadir las columnas seleccionadas al df_cleaned
for col in columns_to_keep:
    df_cleaned[col] = df_original[col]

In [28]:
# Seleccionar las primeras 5 líneas del DataFrame
df_subset = df_cleaned.head(5)

# Guardar el subset en un archivo CSV
ruta_archivo = './muestras_ejemplo.csv'  # Ruta donde se guardará el archivo
df_subset.to_csv(ruta_archivo, index=False)


In [29]:
# Lista de columnas a eliminar
columns_to_remove = [
    "EnrollmentType", "LastUpdatePostDate",
    "LastUpdatePostDateType", "LastUpdateSubmitDate", "LocationCity",
    "LocationState", "LocationZip", "PrimaryCompletionDate",
    "PrimaryCompletionDateType", "PrimaryOutcomeTimeFrame",
    "SecondaryOutcomeTimeFrame", "StartDateType", "StatusVerifiedDate",
    "StudyFirstPostDate", "StudyFirstPostDateType", "StudyFirstSubmitDate",
    "StudyFirstSubmitQCDate", "VersionHolder"
]

# Agregar columnas que comienzan con 'LastUpdate'
columns_to_remove += [col for col in df_cleaned.columns if  col.startswith('LastUpdate')]

# Eliminar estas columnas del DataFrame
df_cleaned = df_cleaned.drop(columns=columns_to_remove)

In [30]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 68 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ArmGroupDescription              10426 non-null  object 
 1   ArmGroupInterventionName         11067 non-null  object 
 2   ArmGroupLabel                    11662 non-null  object 
 3   ArmGroupType                     9759 non-null   object 
 4   BriefSummary                     13501 non-null  object 
 5   BriefTitle                       13501 non-null  object 
 6   CompletionDate                   12888 non-null  object 
 7   CompletionDateType               12764 non-null  object 
 8   Condition                        13500 non-null  object 
 9   ConditionAncestorId              12202 non-null  object 
 10  ConditionAncestorTerm            12202 non-null  object 
 11  ConditionBrowseBranchAbbrev      13253 non-null  object 
 12  ConditionBrowseBra

In [31]:
# Convertir 'StartDate' a formato de fecha
df_cleaned['StartDate'] = pd.to_datetime(df_cleaned['StartDate'], errors='coerce', format='%B %d, %Y')
df_cleaned['Year'] = df_cleaned['StartDate'].dt.year
# Crear una variable para indicar el año de comienzo
df_cleaned['Year'] = df_cleaned['Year'].dropna().astype(int)

In [32]:
df_cleaned['OverallStatus'].dropna().unique()

array(['Recruiting', 'Unknown status', 'Not yet recruiting', 'Completed',
       'Active, not recruiting', 'Terminated', 'Withdrawn',
       'Enrolling by invitation', 'Suspended',
       'Temporarily not available', 'Available', 'Approved for marketing',
       'No longer available'], dtype=object)

In [33]:
# Crear la nueva columna 'SuccessfulCases'
# 'Successful' se asigna cuando 'OverallStatus' es 'completed', lo que indica que el estudio se completó según lo planeado.
# 'Unsuccessful' se asigna cuando 'OverallStatus' es 'suspended', 'withdrawn', o 'terminated', lo que indica que el estudio no se completó como estaba previsto.
# Otros estados de 'OverallStatus' se manejan según se necesite, aquí se asigna 'Other' como categoría general para esos casos.
df_cleaned['SuccessfulCases'] = np.where(
    df_cleaned['OverallStatus'] == 'Completed', 'Successful',
    np.where(
        df_cleaned['OverallStatus'].isin(['Suspended', 'Withdrawn', 'Terminated']),
        'Unsuccessful',
        'Other'
    )
)

In [38]:
df_cleaned['OutcomeAnalysisPValue'] = df_cleaned['OutcomeAnalysisPValue'].replace(to_replace=r'[^\d.]+', value='', regex=True)
df_cleaned['OutcomeAnalysisPValue'] = pd.to_numeric(df_cleaned['OutcomeAnalysisPValue'], errors='coerce')

# Crear la nueva columna 'Significative'
# Se asigna 'Significant' si 'OutcomeAnalysisPValue' es menor que 0.05
# Se asigna 'Non significant' si 'OutcomeAnalysisPValue' es mayor o igual a 0.05
# Se asigna 'Unknown' si 'OutcomeAnalysisPValue' es NaN
df_cleaned['Significance'] = np.where(
    df_cleaned['OutcomeAnalysisPValue'].isna(), 'Unknown',
    np.where(df_cleaned['OutcomeAnalysisPValue'] < 0.05, 'Significant', 'Non significant')
)

In [41]:
from google.colab import files
# Especifica el path del CSV
ruta_archivo = './clinicaltrials_limpio.csv'

# Guardar el DataFrame como CSV
df_cleaned.to_csv(ruta_archivo, index=False, header=True)
files.download('clinicaltrials_limpio.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
print(df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 71 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   ArmGroupDescription              10426 non-null  object        
 1   ArmGroupInterventionName         11067 non-null  object        
 2   ArmGroupLabel                    11662 non-null  object        
 3   ArmGroupType                     9759 non-null   object        
 4   BriefSummary                     13501 non-null  object        
 5   BriefTitle                       13501 non-null  object        
 6   CompletionDate                   12888 non-null  object        
 7   CompletionDateType               12764 non-null  object        
 8   Condition                        13500 non-null  object        
 9   ConditionAncestorId              12202 non-null  object        
 10  ConditionAncestorTerm            12202 non-null  object   